In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
from sklearn import linear_model
import statsmodels.api as sm

### Data Cleaning

In [2]:
crsp = pd.read_sas("crsp_industry_2.sas7bdat", encoding = "ISO-8859-1")
crsp['monthid'] = (crsp.DATE.dt.year-1985)*12 + crsp.DATE.dt.month
crsp['PRC'] = np.abs(crsp['PRC'])
crsp['MktCap'] = crsp['SHROUT']*crsp['PRC']/1000	

Clean the data, make sure each stock has the data from 1985 to 2022

In [3]:
permno = pd.merge(crsp[crsp['DATE'] == '1983-03-31'], crsp[crsp['DATE'] == '2019-12-31'], on='PERMNO', how='inner')['PERMNO']
crsp = pd.merge(crsp, permno, on = ['PERMNO'])

Market Cap > $100 M

In [4]:
# training sample 333 months. testing, start from 2012 Jan
validMktCap = crsp[(crsp['DATE'].dt.month == 1) & (crsp['DATE'].dt.year >= 2010) & (crsp['MktCap'] >= 100)][['PERMNO']].drop_duplicates()
crsp = pd.merge(crsp, validMktCap, on=['PERMNO'])

Add industry

In [5]:
indID = crsp[crsp['DATE'] == '2019-12-31'][['PERMNO', 'NAICS']]
indID['indID'] = indID['NAICS'].str.extract(r'(\d\d)')
crsp = pd.merge(crsp, indID[['PERMNO', 'indID']], on=['PERMNO'])

### Add factors

In [6]:
permno_gvkey = pd.read_sas("permno_gvkey.sas7bdat", encoding = "ISO-8859-1")
crsp = pd.merge(crsp, permno_gvkey[['PERMNO', 'GVKEY']], on = ['PERMNO'], how = 'inner')


In [7]:
# Assuming 'GVKEY' is the column you want to work with
gvkeys = crsp['GVKEY'].copy().drop_duplicates()

# Save to CSV
# gvkeys.to_csv('gvkey.txt', index=False, header=False)


add factors

In [8]:
factors = pd.read_sas("completeFac.sas7bdat", encoding = "ISO-8859-1")
factors['monthid'] = (factors.DATADATE.dt.year-1985)*12 + factors.DATADATE.dt.month

In [9]:
comp_crsp = sqldf("SELECT a.GVKEY, a.monthid, a.PRC, a.VOL, a.RET, a.SHROUT, a.indID, a.MktCap,\
                  b.GVKEY AS GVKEY_2, b.monthid AS monthid_2,\
                  b.ACTQ, b.AJEXQ, b.ATQ, b.CHEQ, b.COGSQ, b.CSHOQ, b.CSHPRQ, b.DLCQ, b.DLTTQ,\
                  b.DVPQ, b.INVTQ, b.LCTQ, b.LTQ, b.NIQ, b.OIADPQ, b.OIBDPQ, \
                  b.PSTKQ, b.REVTQ, b.SEQQ,\
                  b.TXTQ, b.CEQQ\
                  FROM crsp as a\
                  LEFT JOIN factors as b\
                  ON a.GVKEY = b.GVKEY and a.monthid >= b.monthid+4 and a.monthid <= b.monthid+6")

In [10]:
comp_crsp = comp_crsp.dropna()
comp_crsp.sort_values(["GVKEY", "monthid"])
comp_crsp

,GVKEY,monthid,PRC,VOL,RET,SHROUT,indID,MktCap,GVKEY_2,monthid_2,...,LCTQ,LTQ,NIQ,OIADPQ,OIBDPQ,PSTKQ,REVTQ,SEQQ,TXTQ,CEQQ
66,001300,43,35.500000,61092.0,0.028986,149949.0,42,5323.189500,001300,39.0,...,3303.000,6936.000,112.000,220.000,326.000,0.0,2942.000,3191.000,43.000,3191.000
67,001300,44,33.000000,126959.0,-0.057746,149949.0,42,4948.317000,001300,39.0,...,3303.000,6936.000,112.000,220.000,326.000,0.0,2942.000,3191.000,43.000,3191.000
68,001300,45,33.750000,45305.0,0.022727,149949.0,42,5060.778750,001300,39.0,...,3303.000,6936.000,112.000,220.000,326.000,0.0,2942.000,3191.000,43.000,3191.000
69,001300,46,34.125000,54970.0,0.011111,149949.0,42,5117.009625,001300,42.0,...,3442.000,7052.000,128.000,239.000,349.000,0.0,3096.000,3245.000,40.000,3245.000
70,001300,47,34.125000,45397.0,0.013187,149949.0,42,5117.009625,001300,42.0,...,3442.000,7052.000,128.000,239.000,349.000,0.0,3096.000,3245.000,40.000,3245.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117573,011600,417,36.049999,60992.0,0.045821,54871.0,21,1978.099508,011600,413.0,...,698.020,1562.402,37.738,38.140,62.099,0.0,938.842,831.246,9.151,831.246
117574,011600,418,36.810001,44614.0,0.021082,55924.0,21,2058.562517,011600,413.0,...,698.020,1562.402,37.738,38.140,62.099,0.0,938.842,831.246,9.151,831.246
117575,011600,419,38.299999,28758.0,0.040478,55924.0,21,2141.889157,011600,413.0,...,698.020,1562.402,37.738,38.140,62.099,0.0,938.842,831.246,9.151,831.246
117576,011600,420,42.180000,58234.0,0.107572,55094.0,21,2323.864937,011600,416.0,...,482.674,1475.191,-4.776,13.668,37.845,0.0,838.659,787.973,-0.185,787.973


In [11]:
facDF = comp_crsp[['GVKEY', 'indID', 'monthid', 'RET']].copy()
facDF = facDF[facDF['monthid']>0]

# Tobin Q
facDF['ToQ'] = (comp_crsp['MktCap'] + comp_crsp['PSTKQ'] + comp_crsp['LCTQ']-comp_crsp['ACTQ'] + comp_crsp['DLTTQ'])/comp_crsp['ATQ']

# STO
facDF['STO'] = ((comp_crsp['PRC'] - comp_crsp['PRC'].rolling(window=14).min()) / (comp_crsp['PRC'].rolling(window=14).max() - comp_crsp['PRC'].rolling(window=14).min())) * 100

# OBV
facDF['OBV'] = 0
comp_crsp['Change'] = comp_crsp.groupby('GVKEY')['PRC'].diff()
comp_crsp['Volume_Change'] = np.where(comp_crsp['Change'] > 0, comp_crsp['VOL'],  # Price increased
                                  np.where(comp_crsp['Change'] < 0, -comp_crsp['VOL'], 0))  # Price decreased 
facDF['OBV'] = comp_crsp.groupby('GVKEY')['Volume_Change'].cumsum().shift().fillna(0)                                                 

# 1.GPOA
facDF['GPOA'] = (comp_crsp['REVTQ'] - comp_crsp['COGSQ']) / comp_crsp['ATQ']

# 2. ROA
facDF['ROA'] = comp_crsp['NIQ'] / comp_crsp['ATQ']

# 17. OP
facDF['OP'] = comp_crsp['OIADPQ'] / comp_crsp['REVTQ']

# 18. DE
facDF['DE'] = (comp_crsp['LCTQ'] - comp_crsp['ACTQ'] + comp_crsp['DLTTQ']) / comp_crsp['LTQ']

# 21. Dupont
facDF['Dup'] = (comp_crsp['NIQ'] / comp_crsp['REVTQ']) * (comp_crsp['REVTQ'] / comp_crsp['ATQ']) * (comp_crsp['ATQ'] / comp_crsp['SEQQ'])

# 28. Liq
facDF['LIQ'] = (comp_crsp['LCTQ'] - comp_crsp['ACTQ'] + comp_crsp['DLTTQ'] - comp_crsp['CHEQ']) / (comp_crsp['ATQ'] - comp_crsp['INVTQ'])

# Asset to Market
facDF['AM'] = (comp_crsp['ATQ'].shift(12) / (comp_crsp['CSHPRQ'].shift(12) * comp_crsp['AJEXQ'].shift(12))) / comp_crsp['MktCap'].shift(12)

# Debt to Market
facDF['DM']= (comp_crsp['DLCQ'].shift(12) + comp_crsp['DLTTQ'].shift(12)) /  (comp_crsp['PRC'] * comp_crsp['SHROUT']).shift(12)

# TES
facDF['TES'] = comp_crsp['TXTQ'] / (comp_crsp['CSHPRQ'] * comp_crsp['AJEXQ']) - comp_crsp['TXTQ'].shift(12) / (comp_crsp['CSHPRQ'].shift(12) * comp_crsp['AJEXQ'].shift(12))
facDF['TES'] /= comp_crsp['ATQ'].shift(12) / (comp_crsp['CSHPRQ'].shift(12) * comp_crsp['AJEXQ'].shift(12))

# NSI
facDF['NSI'] = np.log((comp_crsp['CSHOQ'].shift(12) *  comp_crsp['AJEXQ'].shift(12)) / (comp_crsp['CSHOQ'].shift(24) *  comp_crsp['AJEXQ'].shift(24)))

# NOA
facDF["NOA"] = (comp_crsp['ATQ']-comp_crsp['CHEQ']) - (comp_crsp['ATQ']-comp_crsp['DLCQ']-comp_crsp['DLTTQ']-comp_crsp['PSTKQ']-comp_crsp['CEQQ'])
facDF["DNOA"] = (facDF["NOA"] - facDF["NOA"].shift(12))/comp_crsp['ATQ'].shift(12)


Fac = ["ToQ", "STO", "OBV", "GPOA", "ROA", "OP", "Dup", "AM", "DM", "TES", "NSI", "DNOA", "LIQ"]

In [12]:
facDF = facDF.dropna()
# facDF

In [13]:
# df = facDF[Fac]
# df.corr()

### Uniformization and data filtering

In [14]:
for month, gMonth in facDF.groupby(['monthid']):
        # for each industry
        for ind, gInd in gMonth.groupby(['indID']):
            # ind[0] is the ind id
            for fac in Fac:
                ind_avg = gInd[fac].mean()
                ind_sd = gInd[fac].std()
                for i in gInd.index:
                    facDF.at[i, f'{fac}_z'] = (facDF.at[i, fac] - ind_avg)/ind_sd
    
facDF

/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/1870076502.py:9: RuntimeWarning: invalid value encountered in scalar divide
  facDF.at[i, f'{fac}_z'] = (facDF.at[i, fac] - ind_avg)/ind_sd
/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/1870076502.py:9: RuntimeWarning: invalid value encountered in scalar divide
  facDF.at[i, f'{fac}_z'] = (facDF.at[i, fac] - ind_avg)/ind_sd
/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/1870076502.py:9: RuntimeWarning: invalid value encountered in scalar divide
  facDF.at[i, f'{fac}_z'] = (facDF.at[i, fac] - ind_avg)/ind_sd
/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/1870076502.py:9: RuntimeWarning: invalid value encountered in scalar divide
  facDF.at[i, f'{fac}_z'] = (facDF.at[i, fac] - ind_avg)/ind_sd
/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/1870076502.py:9: RuntimeWarning: invalid value encountered in scalar divide
  facDF.at[i, f'{fac}_z'] = (facDF.at[i, fac

,GVKEY,indID,monthid,RET,ToQ,STO,OBV,GPOA,ROA,OP,...,GPOA_z,ROA_z,OP_z,Dup_z,AM_z,DM_z,TES_z,NSI_z,DNOA_z,LIQ_z
90,001300,42,67,-0.081633,0.581505,10.526316,19934.0,0.067104,0.012637,0.073977,...,-1.557077,-0.176404,0.121894,0.387704,-0.392115,-0.136332,-0.396446,-0.276421,-0.147535,0.756999
91,001300,42,68,-0.068148,0.542712,0.000000,-19982.0,0.067104,0.012637,0.073977,...,-1.554294,-0.280412,0.049507,0.301592,-0.522198,1.344439,-0.399838,0.381737,-0.326971,0.748164
92,001300,42,69,-0.100806,0.483107,0.000000,-83420.0,0.067104,0.012637,0.073977,...,-1.554294,-0.280412,0.049507,0.301592,-0.513305,1.490557,-0.399838,0.381737,-0.326971,0.748164
93,001300,42,70,-0.098655,0.454066,0.000000,-120041.0,0.064823,0.011620,0.066897,...,-1.603602,-0.640115,-0.408130,-0.308778,-0.501023,1.188311,-0.251571,0.641758,-0.508239,0.702979
94,001300,42,71,0.122388,0.488934,22.105263,-174233.0,0.064823,0.011620,0.066897,...,-1.603602,-0.640115,-0.408130,-0.308778,-0.492057,1.243244,-0.251571,0.643344,-0.508239,0.702979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117573,011600,21,417,0.045821,0.839798,15.353694,766096.0,0.059727,0.015030,0.040625,...,1.537402,0.927670,-0.909648,1.463546,2.435809,0.334244,1.278639,-2.131184,-0.986739,-0.538390
117574,011600,21,418,0.021082,0.871845,28.958803,827088.0,0.059727,0.015030,0.040625,...,1.421783,0.832054,-0.274579,1.166353,2.096939,0.020468,1.158598,-1.926422,-0.526719,-0.746729
117575,011600,21,419,0.040478,0.905033,53.746756,871702.0,0.059727,0.015030,0.040625,...,1.421783,0.832054,-0.274579,1.166353,1.972542,-0.054692,1.158598,-1.926422,-0.526719,-0.746729
117576,011600,21,420,0.107572,1.051015,100.000000,900460.0,0.054002,-0.002005,0.016297,...,1.147252,-0.229656,-0.431998,-0.285407,1.833477,-0.030054,-1.156076,-1.878359,-0.588379,-0.412741


In [15]:
facDF['RET_t1'] = facDF['RET'].shift(-1)
# facDF

### Set training period as 300 month

In [16]:
ret_matrix = sqldf("SELECT i.GVKEY, i.monthid,\
                   j.monthid as monthid2, j.RET_t1, \
                   j.ToQ_z as ToQ,\
                   j.STO_z as STO, j.OBV_z as OBV,\
                   j.GPOA_z as GPOA, j.ROA_z as ROA,\
                   j.OP_z as OP, j.Dup_z as Dup, \
                   j.AM_z as AM, j.DM_z as DM, j.TES_z as TES,\
                   j.NSI_z as NSI, j.DNOA_z as DNOA,\
                   j.LIQ_z as LIQ\
                   FROM facDF AS i, facDF AS j\
                   WHERE i.GVKEY = j.GVKEY AND i.monthid>j.monthid AND i.monthid <=j.monthid+300")

In [17]:
ret_matrix = ret_matrix.dropna()
ret_matrix = ret_matrix[ret_matrix['monthid'] > 300]
ret_matrix = ret_matrix.sort_values(['GVKEY', 'monthid', 'monthid2'])

### Fama MacBeth

In [18]:
Fac = ["STO", "OBV", "AM", "DM", "Dup", "ToQ", "TES", "GPOA","DNOA", "NSI", "LIQ"]

In [19]:
test_beta = {'GVKEY':[], 'monthid':[], 'STO_beta':[], 'OBV_beta':[], 'AM_beta':[], 'DM_beta':[],
        'Dup_beta':[], 'ToQ_beta':[], 'TES_beta':[],
        'GPOA_beta':[], "DNOA_beta":[], "NSI_beta":[], 'LIQ_beta':[]}


for name, group in ret_matrix.groupby(['GVKEY', 'monthid']):
    stage1Test = linear_model.LinearRegression().fit(group[["STO", "OBV", "AM", "DM", "Dup", "ToQ", "TES", "GPOA", "DNOA", "NSI", "LIQ"]], group["RET_t1"])
    test_beta['GVKEY'].append(group['GVKEY'].iloc[0])
    test_beta['monthid'].append(group['monthid'].iloc[0])
    test_beta['STO_beta'].append(stage1Test.coef_[0])
    test_beta['OBV_beta'].append(stage1Test.coef_[1])
    test_beta['AM_beta'].append(stage1Test.coef_[2])
    test_beta['DM_beta'].append(stage1Test.coef_[3])
    test_beta['Dup_beta'].append(stage1Test.coef_[4])
    test_beta['ToQ_beta'].append(stage1Test.coef_[5])
    test_beta['TES_beta'].append(stage1Test.coef_[6])
    test_beta['GPOA_beta'].append(stage1Test.coef_[7])
    test_beta['DNOA_beta'].append(stage1Test.coef_[8])
    test_beta['NSI_beta'].append(stage1Test.coef_[9])
    test_beta['LIQ_beta'].append(stage1Test.coef_[10])
    

test_betaDF = pd.DataFrame(test_beta)

# test_betaDF


In [20]:
test1DF = pd.merge(test_betaDF, facDF[['GVKEY', 'monthid', 'RET_t1']], on=['GVKEY', 'monthid'], how='inner')
test1DF = test1DF.dropna()
# test1DF

In [21]:
test_lamda = {'GVKEY':[], 'monthid':[], 
              'STO_lamda':[], 'OBV_lamda':[],
            'AM_lamda':[], 'DM_lamda':[],
        'Dup_lamda':[], 'ToQ_lamda':[], 'TES_lamda':[],
        'GPOA_lamda':[], 'DNOA_lamda':[], 'NSI_lamda':[], 'LIQ_lamda':[]}

for name, group in test1DF.groupby(['monthid']):
    stage2Test = linear_model.LinearRegression().fit(group[[ "STO_beta", "OBV_beta", "AM_beta", "DM_beta", "Dup_beta", "ToQ_beta", "TES_beta", "GPOA_beta", "DNOA_beta", "NSI_beta", "LIQ_beta"]], group["RET_t1"])
    test_lamda['GVKEY'].append(group['GVKEY'].iloc[0])
    test_lamda['monthid'].append(group['monthid'].iloc[0])
    test_lamda['STO_lamda'].append(stage2Test.coef_[0])
    test_lamda['OBV_lamda'].append(stage2Test.coef_[1])
    test_lamda['AM_lamda'].append(stage2Test.coef_[2])
    test_lamda['DM_lamda'].append(stage2Test.coef_[3])
    test_lamda['Dup_lamda'].append(stage2Test.coef_[4])
    test_lamda['ToQ_lamda'].append(stage2Test.coef_[5])
    test_lamda['TES_lamda'].append(stage2Test.coef_[6])
    test_lamda['GPOA_lamda'].append(stage2Test.coef_[7])
    test_lamda['DNOA_lamda'].append(stage2Test.coef_[8])
    test_lamda['NSI_lamda'].append(stage2Test.coef_[9])
    test_lamda['LIQ_lamda'].append(stage2Test.coef_[10])

test_lamdaDF = pd.DataFrame(test_lamda)
test_lamdaDF

,GVKEY,monthid,STO_lamda,OBV_lamda,AM_lamda,DM_lamda,Dup_lamda,ToQ_lamda,TES_lamda,GPOA_lamda,DNOA_lamda,NSI_lamda,LIQ_lamda
0,001045,301,-0.190298,-0.078562,0.023965,-0.156602,-0.375552,-0.098529,0.134542,0.048558,0.824749,0.234655,0.080515
1,001045,302,-0.700599,0.500606,0.112373,0.016731,-0.082464,-0.012299,-0.025914,-0.013265,-0.153109,0.787985,-0.044942
2,001045,303,-0.203584,0.224077,0.096281,-0.027906,-0.159541,-0.036770,-1.444378,-0.015585,0.654712,0.179393,0.127737
3,001045,304,-0.900992,-0.096335,-0.051207,-0.041405,0.433383,0.167222,-0.186772,0.063383,0.037931,0.353088,-0.071578
4,001045,305,0.954921,-0.050233,-0.159833,-0.169770,0.522980,0.313565,-0.409174,0.113549,-0.118915,-0.291952,0.059660
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,001004,417,-0.319912,0.001233,-0.012141,-0.165341,-0.244063,0.073197,-0.377357,0.350178,0.613998,0.302631,-0.072668
117,001004,418,0.635239,-0.047098,0.031691,0.181082,-0.060982,-0.184626,0.165427,0.206067,-0.335501,0.154154,0.159562
118,001004,419,-0.206227,0.011935,-0.014195,-0.144726,0.240173,0.038474,0.246294,0.179737,0.419444,0.071818,-0.178646
119,001004,420,1.605432,-0.073021,0.009366,-0.188076,0.134620,0.375893,-0.699101,-0.248894,-0.104272,-0.239967,-0.378825


In [22]:
def MacBethStat(col):
    avg = col.mean()
    std = col.std()
    n = len(col)
    tstat = avg/(std/np.sqrt(n))

    return avg, tstat

In [23]:
facTStat = {'Factor':[], 'AVG':[], 'tStat':[], 'abs(tStat)':[]}
for fac in Fac:
    facTStat['Factor'].append(fac)
    avg, t = MacBethStat(test_lamdaDF[f'{fac}_lamda'])
    facTStat['AVG'].append(avg)
    facTStat['tStat'].append(t)
    facTStat['abs(tStat)'].append(np.abs(t))

facTStat = pd.DataFrame(facTStat)
facTStat = facTStat.sort_values(['abs(tStat)'], ascending=[False])
facTStat

,Factor,AVG,tStat,abs(tStat)
6,TES,-0.077205,-1.338477,1.338477
1,OBV,0.012070,1.304835,1.304835
2,AM,0.008316,1.231651,1.231651
10,LIQ,-0.010430,-0.921171,0.921171
0,STO,0.052633,0.904547,0.904547
7,GPOA,0.015453,0.741558,0.741558
8,DNOA,-0.030587,-0.659956,0.659956
9,NSI,-0.026848,-0.582684,0.582684
5,ToQ,0.007989,0.394009,0.394009
3,DM,0.004372,0.329090,0.329090


### Random Forest

In [24]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=15, max_features=7, n_estimators=1, n_jobs=-1)

test_feaImp = {'GVKEY':[], 'monthid':[], 'STO':[], 'OBV':[], 'AM':[], 'DM':[], 'Dup':[], 'ToQ':[], 'TES':[], 'GPOA':[], 'DNOA':[], 'NSI':[], 'LIQ':[]}

for name, group in ret_matrix.groupby(['GVKEY', 'monthid']):
    x_train = group[Fac]
    y_train = group['RET_t1']
    
    rf.fit(x_train, y_train)
    importances = rf.feature_importances_
    test_feaImp['GVKEY'].append(group['GVKEY'].iloc[0])
    test_feaImp['monthid'].append(group['monthid'].iloc[0])
    test_feaImp['STO'].append(importances[0])
    test_feaImp['OBV'].append(importances[1])
    test_feaImp['AM'].append(importances[2])
    test_feaImp['DM'].append(importances[3])
    test_feaImp['Dup'].append(importances[4])
    test_feaImp['ToQ'].append(importances[5])
    test_feaImp['TES'].append(importances[6])
    test_feaImp['GPOA'].append(importances[7])
    test_feaImp['DNOA'].append(importances[8])
    test_feaImp['NSI'].append(importances[9])
    test_feaImp['LIQ'].append(importances[10])


test_feaImpDF = pd.DataFrame(test_feaImp)
# test_feaImpDF

In [25]:
RFfacTStat = {'Factor':[], 'AVG':[]}
for fac in Fac:
    RFfacTStat['Factor'].append(fac)
    RFfacTStat['AVG'].append(test_feaImpDF[fac].mean())

RFfacTStat = pd.DataFrame(RFfacTStat)
RFfacTStat = RFfacTStat.sort_values(['AVG'], ascending=[False])
# RFfacTStat

## After testing factors, training

### Fama MacBeth

In [26]:
selectedFac = ["OBV", "AM", "LIQ", "TES", "STO"]

In [27]:
beta = {'GVKEY':[], 'monthid':[], 'alpha':[], 'OBV_beta':[], 'AM_beta':[],'LIQ_beta':[], 'TES_beta':[], 'STO_beta':[]}

for name, group in ret_matrix.groupby(['GVKEY', 'monthid']):
    stage1Model = linear_model.LinearRegression().fit(group[["OBV", "AM", "LIQ", "TES", "STO"]], group["RET_t1"])
    beta['GVKEY'].append(group['GVKEY'].iloc[0])
    beta['monthid'].append(group['monthid'].iloc[0])
    beta['alpha'].append(stage1Model.intercept_)
    beta['OBV_beta'].append(stage1Model.coef_[0])
    beta['AM_beta'].append(stage1Model.coef_[1])
    beta['LIQ_beta'].append(stage1Model.coef_[2])
    beta['TES_beta'].append(stage1Model.coef_[3])
    beta['STO_beta'].append(stage1Model.coef_[4])

betaDF = pd.DataFrame(beta)

# betaDF


In [28]:
stage1DF = pd.merge(betaDF, facDF[['GVKEY', 'monthid', 'RET_t1']], on=['GVKEY', 'monthid'], how='inner')
stage1DF = stage1DF.dropna()
# stage1DF

In [29]:
lamda = {'GVKEY':[], 'monthid':[], 'OBV_lamda':[], 'AM_lamda':[], 'LIQ_lamda':[], 'TES_lamda':[], 'STO_lamda':[]}

for name, group in stage1DF.groupby(['monthid']):
    stage2Model = linear_model.LinearRegression().fit(group[["OBV_beta", "AM_beta", "LIQ_beta", "TES_beta", "STO_beta"]], group["RET_t1"])
    lamda['GVKEY'].append(group['GVKEY'].iloc[0])
    lamda['monthid'].append(group['monthid'].iloc[0])
    lamda['OBV_lamda'].append(stage2Model.coef_[0])
    lamda['AM_lamda'].append(stage2Model.coef_[1])
    lamda['LIQ_lamda'].append(stage2Model.coef_[2])
    lamda['TES_lamda'].append(stage2Model.coef_[3])
    lamda['STO_lamda'].append(stage2Model.coef_[4])

lamdaDF = pd.DataFrame(lamda)
# lamdaDF

# Testing

### Fama MacBeth

In [30]:
trainingDF = pd.merge(betaDF, facDF[['GVKEY', 'monthid', 'OBV_z', 'AM_z', 'LIQ_z', 'TES_z', 'STO_z', 'RET']], on=['GVKEY', 'monthid'], how='inner')

In [31]:
# trainingDF

In [32]:
trainingDF['predRet'] = trainingDF['alpha']
for fac in selectedFac:
    trainingDF['predRet'] += trainingDF[f'{fac}_beta'] * trainingDF[f'{fac}_z']

In [33]:
# trainingDF

In [34]:
# ranking
trainingDF['rank'] = trainingDF.groupby(['monthid'])['predRet'].transform(lambda x:pd.qcut(x, 10, labels=False))

top10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in trainingDF[trainingDF['rank']==9].groupby(['monthid']):
    top10['monthid'].append(group['monthid'].iloc[0])
    top10['RET'].append(group['RET'].mean())
    top10['predRet'].append(group['predRet'].mean())

top10DF = pd.DataFrame(top10)

bottom10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in trainingDF[trainingDF['rank']==0].groupby(['monthid']):
    bottom10['monthid'].append(group['monthid'].iloc[0])
    bottom10['RET'].append(group['RET'].mean())
    bottom10['predRet'].append(group['predRet'].mean())

bottom10DF = pd.DataFrame(bottom10)

hedgeDF = pd.merge(top10DF, bottom10DF, on=['monthid'])
hedgeDF['RET'] = hedgeDF['RET_x'] - hedgeDF['RET_y']
hedgeDF['predRet'] = hedgeDF['predRet_x'] - hedgeDF['predRet_y']
hedgeDF = hedgeDF.drop(['RET_x', 'RET_y', 'predRet_x', 'predRet_y'], axis=1)
hedgeDF.to_excel('hedgeDF.xlsx')

### Random Forest

In [35]:
RFtrainingDF = facDF[['GVKEY', 'monthid', 'OBV_z', 'AM_z', 'LIQ_z', 'TES_z', 'STO_z', 'RET', 'RET_t1']].copy()

In [36]:
rf = RandomForestRegressor(max_depth=15, max_features=7, n_estimators=1, n_jobs=-1)
rfFac = ['OBV_z', 'AM_z', 'LIQ_z', 'TES_z', 'STO_z']

rfDF = {'GVKEY':[], 'monthid':[], 'RET':[], 'predRet':[]}

for name, group in RFtrainingDF.groupby(['GVKEY']):
    for i in range(301, 421):
        test = group[group['monthid'] == i]
        if test.empty: continue
        test_x = test[rfFac]

        train = group[(group['monthid'] >= i-300) & (group['monthid'] < i)]
        if train.empty: continue
        train_x = train[rfFac]
        train_y = train['RET_t1']
        
        rf.fit(train_x, train_y)
        
        rfDF['GVKEY'].append(test['GVKEY'].iloc[0])
        rfDF['monthid'].append(test['monthid'].iloc[0])
        rfDF['RET'].append(test['RET'].iloc[0])
        rfDF['predRet'].append(rf.predict(test_x)[0])

rfDF = pd.DataFrame(rfDF)

In [37]:
# ranking
rfDF['rank'] = rfDF.groupby(['monthid'])['predRet'].transform(lambda x:pd.qcut(x, 10, labels=False))

RFtop10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in rfDF[rfDF['rank']==9].groupby(['monthid']):
    RFtop10['monthid'].append(group['monthid'].iloc[0])
    RFtop10['RET'].append(group['RET'].mean())
    RFtop10['predRet'].append(group['predRet'].mean())

RFtop10DF = pd.DataFrame(RFtop10)

RFbottom10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in rfDF[rfDF['rank']==0].groupby(['monthid']):
    RFbottom10['monthid'].append(group['monthid'].iloc[0])
    RFbottom10['RET'].append(group['RET'].mean())
    RFbottom10['predRet'].append(group['predRet'].mean())

RFbottom10DF = pd.DataFrame(RFbottom10)

RFhedgeDF = pd.merge(RFtop10DF, RFbottom10DF, on=['monthid'])
RFhedgeDF['RET'] = RFhedgeDF['RET_x'] - RFhedgeDF['RET_y']
RFhedgeDF['predRet'] = RFhedgeDF['predRet_x'] - RFhedgeDF['predRet_y']
RFhedgeDF = RFhedgeDF.drop(['RET_x', 'RET_y', 'predRet_x', 'predRet_y'], axis=1)

In [38]:
RFhedgeDF.to_excel('RFhedgeDF.xlsx')

### Conduct Performence analysis

In [39]:
ff = pd.read_sas("ff.sas7bdat", encoding = "ISO-8859-1")
ff['monthid'] = (ff.DATEFF.dt.year-1985)*12 + ff.DATEFF.dt.month
ff = ff.drop(['DATEFF'], axis=1)

In [40]:
perfStat = {'Raw_Return':[], 'Sharpe_Ratio':[], 'CAPM_alpha':[], '4Fac_alpha':[], 'Information_Ratio':[]}

portDF = pd.merge(hedgeDF, ff, on=['monthid'], how='inner')
portDF['XRET'] = portDF['RET'] - portDF['RF']
portDF['XMKT'] = portDF['RET'] - (portDF["MKTRF"] + portDF['RF'])

capm = sm.add_constant(portDF["MKTRF"])
CAPMmodel = sm.OLS(portDF['XRET'], capm).fit()
ff4 = sm.add_constant(portDF[["MKTRF", "SMB", "HML", "UMD"]])
ff4model = sm.OLS(portDF['XRET'], ff4).fit()

perfStat['Raw_Return'].append(portDF['RET'].sum())
perfStat['Sharpe_Ratio'].append(12*(portDF['XRET'].mean())/(np.sqrt(12)*portDF['XRET'].std()))
perfStat['CAPM_alpha'].append(CAPMmodel.params[0])
perfStat['4Fac_alpha'].append(ff4model.params[0])
perfStat['Information_Ratio'].append(12*(portDF['XMKT'].mean())/(np.sqrt(12)*portDF['XMKT'].std()))
perfStat = pd.DataFrame(perfStat)


/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/634357488.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  perfStat['CAPM_alpha'].append(CAPMmodel.params[0])
/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/634357488.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  perfStat['4Fac_alpha'].append(ff4model.params[0])


In [41]:
perfStat

,Raw_Return,Sharpe_Ratio,CAPM_alpha,4Fac_alpha,Information_Ratio
0,1.135349,1.149853,0.006359,0.006912,-0.169472


### Random Forest

In [42]:
RFperfStat = {'Raw_Return':[], 'Sharpe_Ratio':[], 'CAPM_alpha':[], '4Fac_alpha':[], 'Information_Ratio':[]}

RFportDF = pd.merge(RFhedgeDF, ff, on=['monthid'], how='inner')
RFportDF['XRET'] = RFportDF['RET'] - RFportDF['RF']
RFportDF['XMKT'] = RFportDF['RET'] - (RFportDF["MKTRF"] + RFportDF['RF'])

RFcapm = sm.add_constant(RFportDF["MKTRF"])
RFCAPMmodel = sm.OLS(RFportDF['XRET'], RFcapm).fit()
RFff4 = sm.add_constant(RFportDF[["MKTRF", "SMB", "HML", "UMD"]])
RFff4model = sm.OLS(RFportDF['XRET'], RFff4).fit()

RFperfStat['Raw_Return'].append(RFportDF['RET'].sum())
RFperfStat['Sharpe_Ratio'].append(12*(RFportDF['XRET'].mean())/(np.sqrt(12)*RFportDF['XRET'].std()))
RFperfStat['CAPM_alpha'].append(RFCAPMmodel.params[0])
RFperfStat['4Fac_alpha'].append(RFff4model.params[0])
RFperfStat['Information_Ratio'].append(12*(RFportDF['XMKT'].mean())/(np.sqrt(12)*RFportDF['XMKT'].std()))
RFperfStat = pd.DataFrame(RFperfStat)
RFperfStat

/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/3782000308.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  RFperfStat['CAPM_alpha'].append(RFCAPMmodel.params[0])
/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/3782000308.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  RFperfStat['4Fac_alpha'].append(RFff4model.params[0])


,Raw_Return,Sharpe_Ratio,CAPM_alpha,4Fac_alpha,Information_Ratio
0,2.761418,2.999171,0.022561,0.023045,0.88732


### Combine portfolio

In [43]:
combineDF = pd.merge(trainingDF[['GVKEY', 'monthid', 'RET', 'predRet']], rfDF[['GVKEY', 'monthid', 'predRet']], on=['GVKEY', 'monthid'], how = 'inner')
combineDF['predRet'] = (combineDF['predRet_x']+combineDF['predRet_y'])/2
combineDF = combineDF.drop(['predRet_x', 'predRet_y'], axis=1)
# combineDF

In [44]:
# ranking
combineDF['rank'] = combineDF.groupby(['monthid'])['predRet'].transform(lambda x:pd.qcut(x, 10, labels=False))

combinetop10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in combineDF[combineDF['rank']==9].groupby(['monthid']):
    combinetop10['monthid'].append(group['monthid'].iloc[0])
    combinetop10['RET'].append(group['RET'].mean())
    combinetop10['predRet'].append(group['predRet'].mean())

combinetop10DF = pd.DataFrame(combinetop10)

combinebottom10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in combineDF[combineDF['rank']==0].groupby(['monthid']):
    combinebottom10['monthid'].append(group['monthid'].iloc[0])
    combinebottom10['RET'].append(group['RET'].mean())
    combinebottom10['predRet'].append(group['predRet'].mean())

combinebottom10DF = pd.DataFrame(combinebottom10)

combinehedgeDF = pd.merge(combinetop10DF, combinebottom10DF, on=['monthid'])
combinehedgeDF['RET'] = combinehedgeDF['RET_x'] - combinehedgeDF['RET_y']
combinehedgeDF['predRet'] = combinehedgeDF['predRet_x'] - combinehedgeDF['predRet_y']
combinehedgeDF = combinehedgeDF.drop(['RET_x', 'RET_y', 'predRet_x', 'predRet_y'], axis=1)

In [45]:
combinehedgeDF.to_excel('combinehedgeDF.xlsx')

In [46]:
combineperfStat = {'Raw_Return':[], 'Sharpe_Ratio':[], 'CAPM_alpha':[], '4Fac_alpha':[], 'Information_Ratio':[]}

combineportDF = pd.merge(combinehedgeDF, ff, on=['monthid'], how='inner')
combineportDF['XRET'] = combineportDF['RET'] - combineportDF['RF']
combineportDF['XMKT'] = combineportDF['RET'] - (combineportDF["MKTRF"] + combineportDF['RF'])

combinecapm = sm.add_constant(combineportDF["MKTRF"])
combineCAPMmodel = sm.OLS(combineportDF['XRET'], combinecapm).fit()
combineff4 = sm.add_constant(combineportDF[["MKTRF", "SMB", "HML", "UMD"]])
combineff4model = sm.OLS(combineportDF['XRET'], combineff4).fit()

combineperfStat['Raw_Return'].append(combineportDF['RET'].sum())
combineperfStat['Sharpe_Ratio'].append(12*(combineportDF['XRET'].mean())/(np.sqrt(12)*combineportDF['XRET'].std()))
combineperfStat['CAPM_alpha'].append(combineCAPMmodel.params[0])
combineperfStat['4Fac_alpha'].append(combineff4model.params[0])
combineperfStat['Information_Ratio'].append(12*(combineportDF['XMKT'].mean())/(np.sqrt(12)*combineportDF['XMKT'].std()))
combineperfStat = pd.DataFrame(combineperfStat)
combineperfStat

/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/1861858913.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combineperfStat['CAPM_alpha'].append(combineCAPMmodel.params[0])
/var/folders/30/3wzgw_b57cbd976yz4p29qvwc0glxz/T/ipykernel_2842/1861858913.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combineperfStat['4Fac_alpha'].append(combineff4model.params[0])


,Raw_Return,Sharpe_Ratio,CAPM_alpha,4Fac_alpha,Information_Ratio
0,3.010166,3.18382,0.023705,0.024835,1.098948


### Plot

In [47]:
strategyDF = pd.merge(portDF[['monthid', 'RET', 'MKTRF', 'RF']], RFhedgeDF[['monthid', 'RET']], on=['monthid'])
strategyDF.rename(columns={'RET_x': 'RET_linear'}, inplace=True)
strategyDF.rename(columns={'RET_y': 'RET_RF'}, inplace=True)
strategyDF = pd.merge(strategyDF, combinehedgeDF[['monthid', 'RET']], on=['monthid'])
strategyDF.rename(columns={'RET': 'RET_combined'}, inplace=True)
strategyDF['MKT'] = strategyDF['MKTRF'] + strategyDF['RF']
# strategyDF

In [48]:
strategyDF['yyyymm'] = strategyDF['monthid'].apply(lambda x: f"{1985 + (x - 1) // 12}{(x - 1) % 12 + 1:02d}")
# strategyDF

In [49]:
strategyDF.to_excel('plot.xlsx')